In [1]:
import pandas as pd
import pytz
from datetime import datetime,timedelta
import numpy as np
from pandas.tseries.offsets import BDay

In [2]:
import cx_Oracle
conn_str = 'erp_user/erp_pass@//sun2002-vip.daytonoh.ncr.com:9007/ERPPROD'
conn = cx_Oracle.connect(conn_str)

In [3]:
fd = open('./sql/sql_openorder_usi.sql', 'r')
sql_file = fd.read()
fd.close()

data_usi = pd.read_sql(sql_file, conn)

In [4]:

d = datetime.now(pytz.timezone('Asia/Calcutta'))
filedate_str = d.strftime('%Y%m%d')

In [5]:
# tmpl = pd.read_csv('./map/convert_format_gf1.csv', encoding='utf-8', low_memory=False, keep_default_na='')

In [6]:
cols_dt = ['CREATION_DT', 'NEED_BY','PROMISE_DT','APPROVED_DATE']
for col in data_usi[cols_dt]:
    data_usi[col] = pd.to_datetime(data_usi[col], format='%d-%b-%Y').dt.date

In [7]:
# Fields for Open PO (common for Ennoconn, Jabil)
data_usi['File'] = 'USI'
data_usi['SO'] = ''
data_usi['Booked Dt'] = data_usi['APPROVED_DATE'] # or CREATION_DATE
data_usi['PID'] = data_usi['ITEM']
data_usi['Qty'] = data_usi['QUANTITY']
data_usi['PO'] = data_usi['PO_NUMBER']
data_usi['PROMISE_DT'] = data_usi['PROMISE_DT'].astype('datetime64[ns]')
# data_usi['Ship Dt'] = data_usi['PROMISE_DT'] # is this Ship Dt
# data_usi['Ship Dt'] = data_usi['NEED_BY'] # is this Ship Dt
data_usi['NEED_BY'] = data_usi['NEED_BY'].astype('datetime64[ns]')
data_usi['PROMISE_DT'] = np.where(data_usi['PROMISE_DT'].notnull(),pd.to_datetime(data_usi['PROMISE_DT']) -BDay(5),data_usi['PROMISE_DT'])
data_usi['PD'] = data_usi['PROMISE_DT']
data_usi['Request Dt'] = data_usi['NEED_BY']
data_usi['Tot Qty'] = data_usi['QUANTITY']
data_usi['Org PD'] = data_usi['PROMISE_DT']
data_usi['Critical'] = ''
data_usi['Revenue'] = ''
data_usi['Priority'] = ''
data_usi['Comments-1'] = ''
data_usi['Comments-2'] = ''
data_usi['Customer'] = ''
data_usi['Prod Cate'] = ''

In [8]:
# Fields for Workbench
data_usi['Demand'] = 'PO-USI'
data_usi['DEMAND VAL'] = data_usi['PO_NUMBER']
conditions = [(data_usi['PID'].str.contains('M')==True) & (data_usi['PID'].str.contains('MCC')==False) , data_usi['PID'].str.contains('K')==True,\
                                                  (data_usi['PID'].str.contains('MCC')==True)]
values = ['M','K','MCC']
data_usi['PID Type'] = np.select(conditions, values, default='')
data_usi['Supply Qty'] = data_usi['QUANTITY']
data_usi['Supply Detail'] = data_usi['PO_NUMBER']
data_usi['Whse'] = 'USI'
data_usi['Ship Date'] = data_usi['PROMISE_DT']
data_usi['PD_PO'] = data_usi['PROMISE_DT']
data_usi['IO SSD'] = data_usi['PROMISE_DT']
data_usi['Plant Ship date'] = data_usi['PROMISE_DT']
data_usi['NEED BY DATE'] = data_usi['NEED_BY']
data_usi['IO BOOKED DATE'] = data_usi['APPROVED_DATE']
data_usi['Plant SSD'] = np.where(data_usi['PROMISE_DT'].notnull(), data_usi['PROMISE_DT'], data_usi['NEED_BY'])
data_usi['Plant SSD'] = data_usi['Plant SSD'].astype('datetime64[ns]')
data_usi['Ship Dt'] = data_usi['Plant SSD'] # is this Ship Dt
data_usi['Validation Flag'] = np.where(data_usi['PROMISE_DT'].notnull(), 'PD OK', 'NPD')
data_usi['Supply Type'] = np.where(data_usi['PROMISE_DT'].notnull(), 'PD', 'PO')
data_usi['Supplier'] = 'UNIVERSAL SCIENTIFIC INDUSTRIAL DE MEXICO S.A. DE C.V'
data_usi = data_usi[data_usi['PID']!='']

In [10]:
# Net out in transit shipments
# data_cmp = pd.read_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\completions\completions_usi.csv', \
#                        encoding='utf-8', low_memory=False, keep_default_na='')
# data_cmp = data_cmp[data_cmp['Status'].str.contains('transit') & data_cmp['PO'].str.startswith('50')]

# data_usi['QUANTITY_wIT'] = data_usi['QUANTITY']
# data_cmp = data_cmp.groupby(['PO','MODEL'])['QTY'].sum().reset_index()
# data_cmp.rename(columns={'PO':'PO_', 'MODEL':'MODEL_', 'QTY':'QTY_'}, inplace=True)

# data_usi = pd.merge(data_usi, data_cmp, how='left', left_on=['PO_NUMBER','ITEM'], right_on=['PO_','MODEL_'])
data_usi['Qty'] = np.where( (data_usi['QUANTITY_RECEIVED']>=data_usi['QUANTITY_BILLED']), \
                              data_usi['QUANTITY'], \
                               data_usi['QUANTITY'] - (data_usi['QUANTITY_BILLED']-data_usi['QUANTITY_RECEIVED']) )
data_usi['Supply Qty'] = data_usi['Qty']

In [11]:
data_usi.to_csv(r'E:/_outputs/orders/snd/openpo_usi_v2_' + filedate_str + '.csv', index=False)
# filedate_str = '20230207'
# data_usi.to_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\usi\openpo_usi_v2_' + filedate_str + '.csv', index=False)

In [12]:
data_usi.to_csv('E:/_outputs/orders/snd/workbench_usi_v2.csv', index=False)